In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
(xtrain2, ytrain2), (xtest2, ytest2) = datasets.cifar10.load_data()
xtrain2, xtest2 = xtrain2 / 255.0, xtest2 / 255.0
from keras.preprocessing.image import img_to_array, array_to_img



In [ ]:
from tensorflow.keras.applications import ResNet50
input_layer=layers.Input(shape=(32,32,3))
model_ResNet50=ResNet50(weights='imagenet',input_tensor=input_layer,include_top=False)

model_ResNet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
for layer in model_ResNet50.layers:
    layer.trainable = False
model = models.Sequential()
model.add(model_ResNet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(xtrain2, ytrain2,batch_size=128, epochs=10,
                    validation_data=(xtest2, ytest2))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


391/391 [==============================] - 17s 29ms/step - loss: 2.1510 - accuracy: 0.2174 - val_loss: 2.0128 - val_accuracy: 0.2723
Epoch 2/10
391/391 [==============================] - 9s 22ms/step - loss: 1.9800 - accuracy: 0.2893 - val_loss: 1.9181 - val_accuracy: 0.3196
Epoch 3/10
391/391 [==============================] - 9s 24ms/step - loss: 1.9194 - accuracy: 0.3145 - val_loss: 1.8950 - val_accuracy: 0.3217
Epoch 4/10
391/391 [==============================] - 8s 20ms/step - loss: 1.8838 - accuracy: 0.3263 - val_loss: 1.8675 - val_accuracy: 0.3250
Epoch 5/10
391/391 [==============================] - 8s 21ms/step - loss: 1.8557 - accuracy: 0.3404 - val_loss: 1.8365 - val_accuracy: 0.3572
Epoch 6/10
391/391 [==============================] - 8s 20ms/step - loss: 1.8319 - accuracy: 0.3522 - val_loss: 1.8149 - val_accuracy: 0.3612
Epoch 7/10
391/391 [==============================] - 8s 21ms/step - loss: 1.8170 - accuracy: 0.3572 - val_loss: 1.8039 - val_accuracy: 0.3527
Epoch 8/1

In [ ]:
xtrain2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in xtrain2])
xtest2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in xtest2])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense

def identity_block(X, filters):
    # Implementation of the identity block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X

def convolutional_block(X, filters, s=2):
    # Implementation of the convolutional block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X

def ResNet50(input_shape=(64, 64, 3), classes=10):
    # Implementation of the ResNet-50 architecture

    X_input = Input(input_shape)

    # Zero-padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters=[64, 64, 256], s=1)
    X = identity_block(X, filters=[64, 64, 256])
    X = identity_block(X, filters=[64, 64, 256])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 1024], s=2)
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 2048], s=2)
    X = identity_block(X, filters=[512, 512, 2048])
    X = identity_block(X, filters=[512, 512, 2048])

    # Average Pooling
    X = AveragePooling2D((2, 2))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

# Create ResNet-50 model with specific input shape
model = ResNet50()

# Display model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 zero_padding2d_1 (ZeroPadd  (None, 70, 70, 3)            0         ['input_2[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv2d_53 (Conv2D)          (None, 32, 32, 64)           9472      ['zero_padding2d_1[0][0]']    
                                                                                                  
 batch_normalization_53 (Ba  (None, 32, 32, 64)           256       ['conv2d_53[0][0]']       

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(xtrain2, ytrain2, epochs=10,
                    validation_data=(xtest2, ytest2))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 140s 63ms/step - loss: 2.4906 - accuracy: 0.1293 - val_loss: 2.3271 - val_accuracy: 0.1444
Epoch 2/10
1563/1563 [==============================] - 95s 61ms/step - loss: 2.3801 - accuracy: 0.1425 - val_loss: 4.3735 - val_accuracy: 0.1538
Epoch 3/10
1563/1563 [==============================] - 94s 60ms/step - loss: 2.3712 - accuracy: 0.1477 - val_loss: 2.8825 - val_accuracy: 0.1508
Epoch 4/10
1563/1563 [==============================] - 96s 61ms/step - loss: 2.3370 - accuracy: 0.1481 - val_loss: 2.2572 - val_accuracy: 0.1481
Epoch 5/10
1563/1563 [==============================] - 95s 61ms/step - loss: 2.3477 - accuracy: 0.1455 - val_loss: 2.9803 - val_accuracy: 0.1397
Epoch 6/10
1563/1563 [==============================] - 94s 60ms/step - loss: 2.3183 - accuracy: 0.1496 - val_loss: 2.4958 - val_accuracy: 0.1430
Epoch 7/10
1563/1563 [==============================] - 95s 61ms/step - loss: 2.3006 - accuracy: 0.1540 - val_loss: 7.5427 - val_accur